# Ayudantía 05: Iterables

## Autores: [@tomasgv](https://github.com/tomasgv) & [@manarea](https://github.com/manarea)

# Iterables e Iteradores


### ¿Qué es iterar? 

Se dice que "iteramos" sobre alguna estructura de datos cuando recorremos los elementos que la contienen. Es fácil ver que podemos iterar sobre listas, tuplas, diccionarios, *sets*, entre otros. Lo que tal vez no sabías es que puedes crear tus propias estructuras iterables 😱!! Para esto es necesario entender la diferencia entre **iterable** e **iterador**.

### Iterable
Es cualquier objeto sobre el cual **se puede iterar**. Por ejemplo, se puede aplicar un *for loop* con ellos.

Los iterables **implementan** el método `__iter__`, es decir, se puede hacer `iter(iterable)` o `iterable.__iter__()`.



In [1]:
diccionario = {"uno": 1, "uwu": 2, "hola": "mundo"}
for i in diccionario:
    print(i)
    
print("iter del diccionario retorna:", iter(diccionario))

uno
uwu
hola
iter del diccionario retorna: <dict_keyiterator object at 0x10f44fa70>


### Iterador
Un iterador es un objeto que **itera sobre un iterable**, y es retornado por el método `__iter__()` de un iterable.

Los iteradores también implementan el método `__next__()`, el cual retorna el siguiente elemento de la estructura **cada vez que se invoca**. 

Cuando no quedan más elementos por recorrer, se levanta la excepción `StopIteration`.

In [2]:
iterador = iter(diccionario)

# Notamos que iter(iterador) retorna al mismo iterador
print(f"Soy un {iterador} y mi iter es {iter(iterador)}") 
print(next(iterador))
for i in iterador:    # Se puede iterar sobre ellos, pero...
    print(i)

Soy un <dict_keyiterator object at 0x10f54e830> y mi iter es <dict_keyiterator object at 0x10f54e830>
uno
uwu
hola


In [3]:
iterador = iter(diccionario)

for i in iterador:  # Una vez que se recorre completo, no se 'reinicia' como con los iterables.
    print(i)
print(next(iterador))

uno
uwu
hola


StopIteration: 

**¿Por qué podemos recorrer varias veces un iterable, pero en el caso de un iterador no?**

Para responder a esa pregunta, necesitamos conocer la estructura de un iterable.

A continuación veremos el esqueleto básico de una estructura iterable, creando nuestra propia versión de `range()` (sí, es un iterable).

Primero, definimos nuestro propia clase iterable de rango, con sus atributos y el método `__iter__()`:

In [4]:
class MiIterableRango:
    
    def __init__(self, inicio, fin):
        self.inicio = inicio
        self.actual = inicio
        self.fin = fin
    
    def __iter__(self):
        return MiIterador(self)


Luego definimos nuestro iterador, que debe definir los métodos `__iter__()` y `__next__()`

In [5]:
class MiIterador:
    
    def __init__(self, iterable):
        self.iterable = iterable
    
    def __iter__(self): 
        return self
    
    def __next__(self):
        if self.iterable.actual == self.iterable.fin:
            # Así es como se levanta una excepción del tipo StopIteration
            # con el mensaje "Llegamos al final".
            self.iterable.actual = self.iterable.inicio
            raise StopIteration("Llegamos al final")
        else:           
            valor = self.iterable.actual
            self.iterable.actual += 1
            return valor

Una vez definidas estas clases, probamos instanciar nuestro rango personalizado:

In [6]:
range_bkn = MiIterableRango(0,5)
for i in range_bkn:
    print(i)

0
1
2
3
4


Notamos también que se cumplen las mismas propiedades de iteradores para el método `iter()` de nuestra clase:

In [7]:
iterador = iter(range_bkn)
print(next(iterador))
print(next(iterador))

0
1


**Logramos definir nuestra propia clase de rango!! 🎉🎉**

Como ejercicio, puedes intentar implementar el salto (*skip*) propio de `range()`.

Finalmente, llegamos a una idea un poco compleja:

Los *for loops* llaman al método `__iter__()` de la estructura iterable, y para obtener el siguiente valor en la iteración utilizan el método `__next__()` de la clase es retornada por `__iter__()`.

## Generadores

Son un caso especial de los **iteradores**: nos permiten iterar sobre secuencias de datos sin la necesidad de almacenarlos en alguna estructura especial. En pocas palabras, los generadores simplifican la necesidad de crear clases iterables, pero no son tan personalizables como estos últimos.

Para obtener un generador, se puede usar un formato similar a las listas por comprensión:

In [8]:
lista_comprension = [i for i in range(5)]
print(lista_comprension)

[0, 1, 2, 3, 4]


Si reemplazamos los `[]` con `()`, notaremos que hemos creado un generador:

In [9]:
generador = (i for i in range(5))
type(generador)

generator

Del mismo modo que los iteradores, se pueden recorrer usando `for`, y tampoco se 'reinician' como ocurre con los iterables.

In [10]:
for i in generador:
    print(i)
print(next(generador))

0
1
2
3
4


StopIteration: 

## Funciones Generadoras
Es posible crear funciones que funcionen como generadores en Python, a través de la sentencia `yield`. 

`yield` es similar a `return` en retornar un valor indicado, pero en el momento en que la función se vuelve a llamar, se continúa justo donde se había quedado.

In [11]:
def pares_entre(inicio, fin):
    print(f"Los pares entre {inicio} y {fin} incluyendo límites son:")
    while inicio <= fin:
        if inicio%2 == 0:
            yield inicio
        inicio += 1 # Aquí continuará la función la próxima vez que se llame.
generador = pares_entre(0,8)
type(generador)

generator

In [12]:
for i in generador:
    print(i)

Los pares entre 0 y 8 incluyendo límites son:
0
2
4
6
8


## Enviar valores a funciones generadoras
Es posible interactuar con funciones generadoras y enviarle datos usando el método `send()`.

Este método envía datos a la expresión `yield` de la función, por lo que al hacer algo como `v = yield valor` el valor enviado con `send()` se guardará en la variable `v`


In [13]:
def funcion_generadora_send():
    contador = 0
    while True:
        valor_recibido = yield contador
        print("Hemos recibido {}".format(valor_recibido))
        if valor_recibido is None:  # Consideraremos 0 si nos llega un None
            valor_recibido = 0
        print("Sumaremos {} a nuestro contador".format(valor_recibido))
        contador += valor_recibido  # Sumamos el valor recibido al contador que llevamos

In [14]:
generador_send = funcion_generadora_send()

Para poder enviarle algo a la función, es necesario antes llamar al menos una vez a su método `__next__()`, de modo que se alcance la línea que contiene a la expresión `yield`. 

In [15]:
next(generador_send)
generador_send.send(5)

Hemos recibido 5
Sumaremos 5 a nuestro contador


5

No nos adentraremos mucho en las aplicaciones de este método, pero de todos modos es importante que lo conozcan.

# Funciones de Primera Clase



- Hay una herramienta que quizás no muchos usen, pero ciertamente resulta útil a la hora de programar. Estamos hablando de las funciones de primera clase. Lo que las caracteriza es el hecho de que en solo una línea, se puede definir lo que en otras circunstacias tomaría 10 líneas o más


- Hacer uso de este método de programación no solo es eficiente respecto a la longitud de nuestro código, si no que también es *ordenado*.

    Entonces, imaginemos que en *una tarea*, nuestro código requiere un nivel de cambio considerable. Podemos usar las funciones de primera clase para ahorrarnos un descuento gigante! *#ProTip*




Veamos, *qué tipos repasaremos y como funcionan:*

## Funciones Lambda
Las diferencias que tenemos entre una función normal y una lambda, son las siguientes:

- Retornan algo
- Se defininen como funciones anónimas, de manera que no reciben un nombre y tampoco se hace uso de algún nombre para definirlas o llamarlas, si no que se definen a medida que las vayamos a necesitar.

In [16]:
'''
A continuación, la diferencia se demuestra con un pequeño ejemplo
'''

def operador(numero):
    #definimos una función que busca un numero en nuestra lista
    variable_return = numero ** 2 
    return(variable_return)

nuevo_numero = operador(4)
'''
De este modo, tuvimos que definir una función, para posteriormente
necesitar guardar el resultado de interés en una variable.
Lineas en total = 5 
'''

#En comparación, tenemos nuestra función hecha mediante el método Lambda

numero_de_pana = lambda x: int(x) ** 2 # Esta funcion podría haber recibido más argumentos

'''
Como podemos ver, solo tuvimos que usar 1 línea de código, y además
queda ordenado y explícito.

Es importante explicar que por el momento, solo podemos crear funciones que 
tengan funciones limitadas, pero en breve veremos que nuestro alcance serán
mucho más amplio a la hora de definir con primera clase.
'''

'\nComo podemos ver, solo tuvimos que usar 1 línea de código, y además\nqueda ordenado y explícito.\n\nEs importante explicar que por el momento, solo podemos crear funciones que \ntengan funciones limitadas, pero en breve veremos que nuestro alcance serán\nmucho más amplio a la hora de definir con primera clase.\n'

### Función map
- El método map nos permite usar una función lambda que hayamos creado y usarla en uno o más iterables (listas, sets, etc).
- Es importante saber que este método nos entregará un generador, razón por la que también podremos iterar sobre él.
- Veamos a continuación nuestros nuevos alcances...

In [17]:
'''
Creamos una función que podamos replicar mediante lambda y map.
En este caso, lo que haremos será juntar dos listas
'''
nombres = ["Max", "Tomás", "María", "Jose"]
apellidos = ["Narea", "González", "Carvajal", "Sepúlveda"]

def como_te_llamas(nombres, apellidos):
    lista_todo = list()
    for nombre, apellido in zip(nombres, apellidos) :
        persona = " ".join([nombre, apellido])
        lista_todo.append(persona)
    return(lista_todo)

print(como_te_llamas(nombres, apellidos))

#Ahora procedemos a definir nuestra función en tan solo una línea
nombres_apellidos = map(lambda nombre, apellido: nombre + " " + apellido, nombres, apellidos)
print(list(nombres_apellidos))
#Tuvimos que poner todo en lista, dado que 

'''
Nuevamente, solo usamos una línea. Requiere práctica, pero a 
la larga es muy útil!

Un buen ejemplo sería a la hora de hacer bases de datos o
trabajar con archivos CSV
'''

['Max Narea', 'Tomás González', 'María Carvajal', 'Jose Sepúlveda']
['Max Narea', 'Tomás González', 'María Carvajal', 'Jose Sepúlveda']


'\nNuevamente, solo usamos una línea. Requiere práctica, pero a \nla larga es muy útil!\n\nUn buen ejemplo sería a la hora de hacer bases de datos o\ntrabajar con archivos CSV\n'

### Función filter

Ahora, procederemos a hacer uso de la función filter, mediante ella podemos definir un criterio del tipo *"esto sí", "esto no"*, para categorizar elementos o recuperar de algún iterable los elementos que nos sean útiles o de interés!

In [18]:
'''
Para proceder a usar la filter, necesitaremos un
iterador y una función que retorne SOLO true o false
'''
texto = "Mucho se ha dicho respecto al alcance de la física cuántica en los tiempos contemporáneos. Desde los revolucionarios aportes de Albert Einstein a mediados del siglo, hasta las recientes experiencias con fotones y la aceleración de partículas, nuestro entendimiento del universo ha variado tanto, en sentidos tan impredecibles, que a nadie sorprenderá lo intangible de la discusión teórica involucrada en este ensayo."
texto = texto.replace(",", "").replace(".", "").split(" ")
#print(texto)

def palabras_de_interes(texto):
    palabras_interesantes = list()
    for palabra in texto:
        if len(palabra) > 3:
            palabras_interesantes.append(palabra)
            #Parte no necesaria para efectos de medición de largo
    return(palabras_interesantes)

'''
Nuestro código anterior itera las palabras de un texto cualquiera y 
nos entrega solo aquellas que tengan un largo "interesante". 
Sacamos palabras como "a", "el", "en", etc
'''

#Ahora, en una sola línea sería:

texto_para_un_ramo = filter(lambda palabra: len(palabra) > 3, texto)
print(list(texto_para_un_ramo))

['Mucho', 'dicho', 'respecto', 'alcance', 'física', 'cuántica', 'tiempos', 'contemporáneos', 'Desde', 'revolucionarios', 'aportes', 'Albert', 'Einstein', 'mediados', 'siglo', 'hasta', 'recientes', 'experiencias', 'fotones', 'aceleración', 'partículas', 'nuestro', 'entendimiento', 'universo', 'variado', 'tanto', 'sentidos', 'impredecibles', 'nadie', 'sorprenderá', 'intangible', 'discusión', 'teórica', 'involucrada', 'este', 'ensayo']


### Función reduce

- Este caso en particular, corresponde a aplicar un resultado obtenido mediante una función, al elemento siguiente usando la misma función. De manera que se va "guardando" un resultado parcial.
- En términos prácticos, corresponde a aplicar una función sobre una función, sobre una función....

In [20]:
'''
Si observamos en detalle, nuestra operacion va juntando de manera secuencial
las listas, de forma que el resultado final, corresponde a una lista
con todos los elementos que se van iterando

Para este caso en particular, se va iterando de dos en dos
'''
from functools import reduce

lista_con_listas = [[1, 2], [3, 4], [5, 6], [7, 8, 9]]
lista_aplanada = reduce(lambda x, y: x + y, lista_con_listas)
lista_aplanada

[1, 2, 3, 4, 5, 6, 7, 8, 9]

## Ejercicio Propuesto 2.1: Cambiador de texto
Luego de haber aprendido a usar filter y reduce, se te ocurre darle un uso entretenido, por lo que decides hacer un cambiador de texto. Este recibirá una lista con palabras y lo procesará para dejar en mayúscula todas aquellas palabras que comienzan con la letra a (puedes usarlo para otras letras también).

A continuación se te entrega la lista con palabras:

In [21]:
texto = ["hola", "me", "llamo", "amalia", "y", "me", "gustan", "las", "abejas", "y", "los", "árboles"]


A continuación tendrás que usar filter para identificar las palabras y luego dejar en mayúscula las que empiezan con a:

In [22]:
'''
En primer lugar, separamos las palabras que debemos 
cambiar de las que no usando filter.

Si queremos conservar el orden en la lista final, 
también debemos guardar los índices.

Notamos que tenemos que convertir las funciones map
y filter a lista en el caso que debamos iterar por 
ellas más de una vez, pues se comportan del mismo modo
que los generadores. 
'''
# One-liner de la solución
palabras_cambiadas = list(map(lambda x: x[0].upper() + x[1:] if (x[0] == "a" or x[0] == "á") else x, texto))
# print(palabras_cambiadas)

palabras_con_a_inicial = list(filter(lambda x: x[0] == "a" or x[0] == "á", texto))
otras_palabras = list(filter(lambda x: x[0] != "a", texto)) 

# Luego aplicamos el cambio a mayúsculas sobre las palabras necesarias usando map.

palabras_con_a_inicial =list(map(lambda x: x[0].upper() + x[1:], palabras_con_a_inicial))
print(palabras_con_a_inicial)

['Amalia', 'Abejas', 'Árboles']


Y finalmente reduce, para juntar nuevamente el texto.

In [23]:
from functools import reduce

# Sin mantener orden de palabras
texto = reduce(lambda x, y: x + y, [palabras_con_a_inicial,otras_palabras])
'''
No se pedirá ordenarlas para que se mantenga el sentido,
ya que se complejiza mucho el ejercicio por este camino.

De todos modos, analiza el one-liner de la solución,
que cambia las palabras a medida que avanza por el
texto original.
'''
print(texto)

['Amalia', 'Abejas', 'Árboles', 'hola', 'me', 'llamo', 'y', 'me', 'gustan', 'las', 'y', 'los', 'árboles']


## **Actividad!! 🎉🎉**

¡Han atacado nuevamente al Banco DCC! Esta vez los malévolos *hackers*, en un intento de pasar desapercibidos, decidieron robarle dinero solo a las personas cuyo nombre comienza con F.

Tu misión será obtener las pérdidas totales generadas por este ataque, y luego devolverle el dinero necesario a cada cliente. Te han entregado la clase `Cliente` con sus atributos listos para ayudarte a resolver este problema.

El banco DCC sospecha que lo han atacado mediante los *loops for* y *while*, por lo que te han **prohibido usarlos en tu código**, a menos que se usen en conjunto con funciones generadoras.

Puedes asumir que las personas cuyo nombre comienza con F no han gastado nada de dinero.


In [1]:
class Cliente:
    def __init__(self, nombre, monto_inicial, monto_actual):
        self.nombre = nombre
        self.monto_inicial = monto_inicial
        self.monto_actual = monto_actual
    
    def depositar(self, monto):
        self.monto_actual += monto
        print(f"Se agregaron {monto} a la cuenta")
    

datos = [['Tomás', 2000, 1800], ['Maximiliano', 9999, 9898], ['Fernando', 10000, 6500],
         ['Francisca', 8750, 5000], ['Uwucito', 1312, 420], ['DIO', 9090, 1864],
         ['Daniela', 2000, 1500], ['F', 99999, 0]]


En primer lugar, nos entregan datos como lista, y debemos convertirlos a objetos.

In [2]:
# ¿Qué función nos puede servir?
clientes = map(lambda x: Cliente(x[0], x[1], x[2]), datos)
clientes = list(clientes)

Luego, debemos obtener solo los clientes cuyos nombres comienzan con F.

In [3]:
# ¿Qué función nos puede servir?
clientes_afectados = filter(lambda cliente: cliente.nombre[0] == "F", clientes)
clientes_afectados = list(clientes_afectados)
print(f'Los clientes afectados son: {[cliente.nombre for cliente in clientes_afectados]}')

Los clientes afectados son: ['Fernando', 'Francisca', 'F']


Finalmente, nos piden calcular las diferencias entre montos iniciales y finales de estos clientes.

In [4]:
# ¿Qué función nos puede servir?
from functools import reduce
monto_robado = reduce(lambda x, y: x + y.monto_actual, clientes_afectados, 0)
print(monto_robado)

11500


In [5]:
'''
A continuación, definiremos una función generadora que nos 
permita ir obteniendo los montos que corresponden a nuestros 
clientes afectados.
'''

def calcula_cuentas(clientes_afectados):
    #Función generadora
    for cliente in clientes_afectados:
        monto_a_devolver = cliente.monto_inicial - cliente.monto_actual
        yield monto_a_devolver #nos


#Creamos un generador de los clientes que ayudaremos
generador_clientes = (cliente for cliente in clientes_afectados) 
#instanciamos nuestra función generadora, la cual nos entregará los montos a arreglar
montos_cuentas = calcula_cuentas(clientes_afectados)
for cliente in generador_clientes:
    cliente.depositar(next(montos_cuentas)) #Vamos llamando los montos que necesitamos
    print(f"Le cliente {cliente.nombre} volvió a tener {cliente.monto_actual}") #Revisamos los montos por cliente

Se agregaron 3500 a la cuenta
Le cliente Fernando volvió a tener 10000
Se agregaron 3750 a la cuenta
Le cliente Francisca volvió a tener 8750
Se agregaron 99999 a la cuenta
Le cliente F volvió a tener 99999
